In [ ]:
# src/models.ipynb
"""
Contains all models to be tried for hot dog classifier
Required mounting to drive and import of importnb to work with .ipynv files. Assumed to have already been done in main script.
Required to be in right directory. also assumed to have been done in main script.
"""
#Loading ML libraries
from tensorflow import keras
import tensorflow as tf

## Importing relevant libraries for CNN model construction
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten
from tensorflow.keras.layers import MaxPooling2D, Activation

# pulling variables from config
import json

with open('config.json') as f:
    cfg = json.load(f)

img_rows = cfg["data_parameters"]["img_rows"]
img_cols = cfg["data_parameters"]["img_cols"]
channels = cfg["data_parameters"]["channels"]
num_classes = cfg["data_parameters"]["num_classes"]
activation = cfg["data_parameters"]["activation"]

# Simple ANN
def model_1():
    model = Sequential()
    model.add(Flatten(input_shape=(img_rows, img_cols, channels)))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))
    model.summary()
    return model

# CNN model 1 (build on top of model_1)
def model_2():
    model = Sequential()
    model.add(Conv2D(50, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation=activation))
    model.summary()
    return model

# Did some tweaking with model_2
def model_3():
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3),
                     activation='relu',
                     input_shape=(img_rows, img_cols, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation=activation))
    model.summary()
    return model

# Try building on top of pre-trained model
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D

def model_4():
    # 1. Create the base model with the right input shape
    base_model = VGG16(
        include_top=False,               # exclude the classifier
        weights='imagenet',              # use pretrained weights
        input_shape=(img_rows, img_cols, channels)
    )

    # 2. Freeze base model layers and only train on top of it
    base_model.trainable = False

    # 3. Adding layers on top of base model
    x = base_model.output
    x = GlobalAveragePooling2D()(x)      # flatten features
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(num_classes, activation=activation)(x)

    # 4. final model
    model = Model(inputs=base_model.input, outputs=predictions)

    model.summary()

    return model

# inspired by datacodecamp on yt. this model mirror the parameters in the tutorial.
def model_5():
  # Data augment
  data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])
  model = Sequential()
  model.add(data_augmentation)
  model.add(Conv2D(128, (3, 3), activation='relu', input_shape=(img_rows, img_cols, channels)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(64, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
  model.add(MaxPooling2D((2, 2)))
  model.add(Dropout(0.25))
  model.add(Conv2D(32, (3, 3), activation='relu', kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
  model.add(Flatten())
  model.add(Dense(128, activation='relu'))
  model.add(Dropout(0.25))
  model.add(Dense(num_classes, activation=activation))
  model.summary()

  return model
